In [3]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.optim import Adam
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torch.utils.data import DataLoader

def one_hot_encoding(x, y):
    # در اینجا از ورودی یک کپی میسازیم و بر حسب اندیس های بردار برچسب هایمان، ورودی را مقدار دهی مکنیم و در 10 درایه ی اول ماتریس ورودی به صورت وان هات جایگذاری میکنیم.
    x_hat = x.clone()
    x_hat[range(x.shape[0]), y] = x.max()
    return x_hat
# در این بخش داده هارا نرمالیزه میکنیم . و فلت میکنیم و آنها را به تنسور تبدیل میکنیم. بچ هایی که تعریف کردیم به سایز کل دیتاست ما هستند و تمامی دادگان دیتاست را یکجا پردازش میکنیم.
# دادگان مثبت و منفی را نیز در این تابع تولید مکنیم. دادگان مثبت را با برچسب های درست و دادگان منفی را با استفاده از برچسب های غلط تولید میکنیم.
def Dataloader():
    def normalize_data(x):

        x_min = torch.min(x, dim=1, keepdim=True)[0]
        x_max = torch.max(x, dim=1, keepdim=True)[0]
        x_norm = torch.sub(x, x_min)
        x_norm = torch.div(x_norm, x_max - x_min)
        x_ = torch.flatten(x)
        return x_

    transform = Compose([ToTensor(),Lambda(normalize_data)])

    train_loader = DataLoader(MNIST('./data/', train=True,download=True,transform=transform),batch_size=50000, shuffle=True)
    test_loader = DataLoader(MNIST('./data/', train=False,download=True,transform=transform),batch_size=10000, shuffle=False)
    x, y = next(iter(train_loader))
    x, y = x.cuda(), y.cuda()
    x_test, y_test = next(iter(test_loader))
    x_test, y_test = x_test.cuda(), y_test.cuda()
    x_positive = one_hot_encoding(x, y)
    x_negative = one_hot_encoding(x, y[torch.randperm(x.size(0))])
    return x,y,x_test,y_test,x_positive,x_negative

class AFNet(torch.nn.Module):
# یک شبکه ی سه لایه مخفی تعریف میکنیم که سایز ورودی آن به ابعاد تصویر فلت شده است.

    def __init__(self):
        super().__init__()
        self.layers = []
        self.layers.append (Layer(784, 600).cuda())
        self.layers.append (Layer(600, 400).cuda())
        self.layers.append (Layer(400, 300).cuda())
# با توجه به شبکه ای که تعریف کردیم و کلاس لایه ای که جداگانه تعریف کردیم، بخش آموزش شبکه، ورودی های منفی و مثبت را دریافت میکند و با استفاده از کلاس بعدی آنهارا آموزش میدهید.

    def train(self, positive_data, negative_data):
        out_positive, out_negative = positive_data, negative_data
        for layer in self.layers:
            out_positive, out_negative = layer.train(out_positive, out_negative)
# در اینجا دو لیست تعریف میکنیم که یکی برای معیار خوب بودن در هر لایه است و یکی معیار خوب بودن در نهایت برای هر برچسب است.
    def predict(self, x,y):
        g_label = []
        unique_labels = torch.unique(y)
        label_num = len(unique_labels)
        for label in range(label_num):
            data_gen = one_hot_encoding(x, label)
            g_layer = []
            for layer in self.layers:
                data_gen = layer(data_gen).pow(2)
                g_layer += [data_gen.mean(1)]
            g_label += [sum(g_layer).unsqueeze(1)]
        g_label = torch.cat(g_label, 1)
        label_predict = g_label.argmax(1)
        return label_predict
# از آنجایی که در این روش ما با هر لایه و آموزش تابع هزینه ی محلی آن سر و کار داریم، یک کلاس لایه ی جداگانه تعریف میشود. که خروجی را به صورت لایه به لایه تولید میکند و به کلاس شبکه ی اصلی میدهد.

class Layer(nn.Linear):
    def __init__(self, in_features, out_features,bias=True):
        super().__init__(in_features, out_features, bias)
        self.relu = torch.nn.ReLU()
        self.optimizer = Adam(self.parameters(), lr=0.05)

    def forward(self, x):
        x_norm = torch.nn.functional.normalize(x, p=2, dim=1, eps=1e-5)
        x_out = self.relu(torch.mm(x_norm, self.weight.T) +self.bias.unsqueeze(0))
        return x_out
# در بخش آموزش شبکه با توجه به معیار خوب بودنی که تعریف میشود، و آستانه ای که انتخاب میکنیم، شبکه سعی میکند فعالیت بخش دادگان مثبت را تا جای امکان بالا نگه دارد و در مورد دادگان منفی، برعکس عمل میکند.
# در نهایت هم بعد از آموزش ایپاک های مورد نظر، ورودی از لایه ی مورد نظر رد میشود و به بخش آموزش شبکه ی اصلی داده میشود.
    def train(self, x_positive, x_negative):
        num_epochs = 1000
        threshold = 2.0
        for i in range(num_epochs):
            loss = torch.log(1 + torch.exp(torch.cat([ threshold-(self.forward(x_positive).pow(2).mean(1)),(self.forward(x_negative).pow(2).mean(1))-threshold]))).mean()
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        out_layer_pos = self.forward(x_positive)
        out_layer_neg = self.forward(x_negative)
        return out_layer_pos.detach(),out_layer_neg.detach()

torch.manual_seed(1234)
x_train,y_train,x_test,y_test,positive_data,negative_data = Dataloader()
net = AFNet()
net.train(positive_data, negative_data)

Train error: 0.08424001932144165
Test error: 0.08310002088546753


In [15]:
predictions = net.predict(x_test, y_test)
accuracy_test = (predictions == y_test).sum().item()/len(y_test)

In [10]:
accuracy_test

0.9169

In [13]:
predictions = net.predict(x_train, y_train)
accuracy_train = (predictions == y_train).sum().item()/len(y_train)

In [14]:
accuracy_train

0.91576